In [ ]:
cc.VerficiationHandler.close_browser()

## Time to walk the elements of the "Bertha" mother element
### This has turned out to be a big thing.  It needs to be tackled for sucessful parsing of nathan and tom elements.  
### We do not look for indepentent trees for verified pointers.  We exploit the fact that the nathans and toms are paired by the nature of the verified pointer
### If I have a nathan and I walk a portion of the tree through, the mother element to get to tom, then I should be able to do the same thing for other elements with the same name as the parent element of nathan that are decendants of bertha.


In [1]:
% run contactsScraper.py

Done
Done
KEYS
Done
Done
RECORDS COLLECTED
DATAFRAMES READY
RECORD OUTPUT READY
CONTACT CHECKER READY


In [5]:
import numpy as np
import pandas as pd

In [2]:
orgsForToday = ['National Association for Multi-Ethnicity In Communications (NAMIC)',
                'Association for Women in Science',
                'Brain Injury Association of America',
                'American Society of Home Inspectors',
                'NAADAC, the Association for Addiction Professionals',
                'American Public Transportation Association',
                'Indiana Soybean Alliance',
                'Associated Builders and Contractors (ABC)',
                'National Association of Social Workers',
                'American Marketing Association (AMA)']

In [3]:
org  = orgsForToday[8]

vh = cc.MotherSetVerifier(org)

No Fred
No Fred


In [4]:
cc.VerificationHandler.orgRecords.orgSessionStatusCheck()

['Tue Oct 17, 2017  12:44:08', '4.917', 's', 'Good']

In [6]:
pointers = vh.verifiedPointers

In [7]:
len(pointers)

2

In [8]:
## Grandmother Finding Algorithm

gmElements = []
gmMatrix = []

for i in range(len(pointers)):
    igmElements = []
    for j in range(i):
        ## Check to see if the Any Mother element is a Big Momma or "Bertha" Element
        if pointers[i].get_mother_element() is pointers[j].get_mother_element():
            gm = pointers[i].get_mother_element()
        else:
            gm = pointers[i].common_parent(pointers[j])
        # Append Match to Grand Mother Matrix
        igmElements.append(gm)
        
        # Check to see if this is a new grand mother element,
        # if so append to the gmElements list of unique grandmother elements 
        if gm not in gmElements:
            gmElements.append(gm)
            
    # Append Matrix Row
    gmMatrix.append(igmElements)
    
grandMotherMatrix = np.matrix(gmMatrix)

In [9]:
## Check the number of grandmothers
len(gmElements)

1

In [10]:
type(gmElements[0])

bs4.element.Tag

In [13]:
gm = gmElements[0]

In [11]:
a = pointers[0].get_mother_element()
b = pointers[1].get_mother_element()

In [12]:
a is b

True

In [14]:
b is gm

True

In [15]:
p0 = pointers[0]

In [19]:
p0.nathan.parent

<h3 class="h3_addpading">Patrick J. Johnson, LCSW</h3>

In [20]:
p1 = pointers[1]

In [23]:
p1.nathan.parent.name

'h3'

In [24]:
print(gm.prettify())

<div id="dnn_ctr17530_HtmlModule_lblContent">
 <h2 class="h2_addpading">
  President
 </h2>
 <h3>
  Kathryn Conley Wehrmann, PhD, MSW, LCSW
 </h3>
 <img alt="Kathryn Conley Wehrmann, PhD, MSW, LCSW" src="/portals/0/Images/_About/board-wehrmann.jpg?ver=2017-05-05-113330-207" style="max-width: 100%; float: left; padding-bottom: 20px; padding-right: 15px; padding-top: 15px;" title="Kathryn Conley Wehrmann, PhD, MSW, LCSW"/>
 <p>
  <strong>
   Kathryn Conley Wehrmann, PhD, MSW, LCSW
  </strong>
  (Illinois Chapter), received her PhD and MSW from the University of Illinois, Urbana-Champaign. She is currently a faculty member and BSW Program Director at Illinois State University (ISU). She also served in public child welfare and healthcare settings, providing a sound background for being chosen to be a Senior Fulbright Scholar to Romania where she taught and conducted research on child welfare reform. Dr. Wehrmann served at ISU as principal investigator on two child welfare training grants a

In [25]:
len(list(gm.children))

51

In [27]:
len(list(gm.descendants))

83

#### 'find_all' passes the gm decendants test

In [33]:
gm.find_all('strong') ## These are not direct decendants, two levels down

[<strong>Kathryn Conley Wehrmann, PhD, MSW, LCSW</strong>,
 <strong>Mildred (Mit) Joyner, MSW, LCSW, BCD</strong>,
 <strong>Heather Bradley-Geary, MSW</strong>,
 <strong>Patrick J. Johnson, LCSW </strong>]

In [38]:
gm.find_all('strong')[0].parent.parent is gm

True

In [40]:
p1.nathan.parent.name

'h3'

In [39]:
starts = gm.find_all(p1.nathan.parent.name)

In [41]:
starts  ## Good

[<h3>Kathryn Conley Wehrmann, PhD, MSW, LCSW </h3>,
 <h3 class="h3_addpading"> Mildred (Mit) Joyner, MSW, LCSW, BCD </h3>,
 <h3 class="h3_addpading">Heather Bradley-Geary, MSW</h3>,
 <h3 class="h3_addpading">Patrick J. Johnson, LCSW</h3>]

In [82]:
## Tag tree with pattern from nathan to tom
def tag_nathans(pt):
    return parent_cycle_up(pt.get_mother_element(), pt.nathan.parent, 'nathan', 0)

def tag_toms(pt):
    return parent_cycle_up(pt.get_mother_element(), pt.tom.parent, 'tom', 0)

def parent_cycle_up(motherElement, element, atr, num):
    ## for the bertha case we stop at the element that is right befor the mother elemenet
    if element.parent is motherElement:
        element[atr] = num
        element['sib'] = atr
    else:
        element[atr] = num
        return(motherElement, element.parent, atr, num + 1)
    
def has_sib(tag):
    return 'sib' in tag.attrs

def is_tom_sib(tag):
    return 'sib' in tag.attrs and tag['sib'] == 'tom'

def is_tom_n(tag):
    return 'sib' in tag.attrs and tag['sib'] == 'tom'

def is_tom_shoulder(tag):
    return 'sib' in tag.attrs and tag['sib'] == 'tom'

def is_nathan_sib(tag):
    return 'sib' in tag.attrs and tag['sib'] == 'nathan'

def is_nathan_n(tag):
    return 'sib' in tag.attrs and tag['sib'] == 'nathan'

def is_nathan_shoulder(tag):
    return 'sib' in tag.attrs and 'nathan' in tag.attrs

def tag_sibs(p1):
    tomShoulder = p1.get_mother_element().find(is_tom_shoulder)
    sibNo = 0
    for posSib in tomShoulder.next_siblings:
        if posSib.name:  # Element has to have a name
            sibNo += 1
            posSib['sib'] = sibNo

    
    sibNo = 0
    for negSib in tomShoulder.previous_siblings:
        if negSib.name: # Element has to have a name
            sibNo -= 1
            negSib['sib'] = sibNo



In [79]:
tag_nathans(p1)
tag_toms(p1)
tag_sibs(p1)

In [45]:
## Seach Children By Attribute
p1.tom.parent.attrs

{'class': ['h2_addpading'], 'sib': 'tom', 'tom': 0}

In [63]:
ts=p1.get_mother_element().find(is_tom_shoulder)

In [ ]:
# Tagging the siblings
## Start with the tom shoulder
gm.find(is_tom_shoulder)
## iterate positive to the right

## iterate negative to the left

In [83]:
gm.find(is_tom_shoulder)

<h2 class="h2_addpading" sib="tom" tom="0"> Vice President </h2>

In [65]:
len(list(ts.next_siblings))

38

In [66]:
len(list(ts.previous_siblings))

12

In [84]:
gm.find_all(is_nathan_shoulder)

[<h3 class="h3_addpading" nathan="0" sib="1"> Mildred (Mit) Joyner, MSW, LCSW, BCD </h3>]

In [70]:
type(type(ts))

type

In [74]:
ts.name

'h2'

In [75]:
ts.string

' Vice President '

In [77]:
ts.string.name == None

True